# Beltway Rate remapping Project
# EEG/Oscillation Analysis 2
## Examining how theta (cycles, precession, etc) and possibly gamma interact w rate remapping effect
## Late Feb 2021 WH 

### Imports and console

In [1]:
import numpy as np, matplotlib.pyplot as plt, scipy.signal as ss, scipy.fftpack as sf
from scipy.signal import hilbert, butter, lfilter
import scipy.fftpack
from collections import OrderedDict

import ratterdam_CoreDataStructures as Core
import ratterdam_ParseBehavior as Parse
import ratterdam_Defaults as Def
import ratterdam_visBasic as Vis
import utility_fx as util

In [2]:
%qtconsole --style native

### Function and Structure Definitions

In [3]:
def filter_data(data, low,high, sf=32000, order=2):
    # Determine Nyquist frequency
    nyq = sf/2
    # Set bands
    low = low/nyq
    high = high/nyq
    # Calculate coefficients
    b, a = butter(order, [low, high], btype='band')
    # Filter signal
    filtered_data = lfilter(b, a, data)

    return filtered_data

In [4]:
hilbert3 = lambda x: hilbert(x, scipy.fftpack.next_fast_len(len(x)))[:len(x)] # this truncates bc for certain array sizes (primes) underlying FFT is insanely slow.

In [5]:
ncsType = np.dtype([
    ('ts', '<u8'),
    ('dwChannelNumber', '<u4'),
    ('fs', '<u4'),
    ('NumValidSamples', '<u4'),
    ('data', '<i2', (512, ))
])

In [6]:
alleylongcoord = {16:'x',17:'y',3:'y',4:'x',5:'x',7:'x',8:'y',10:'y',11:'x'}

### Read in data

In [27]:
rat = 'R781'
expCode = 'BRD3'
datafile = f'E:\\Ratterdam\\{rat}\\{rat}{expCode}\\'
clustname = "TT5\\cl-maze1.1"
qualThresh = 3
alleyTracking, alleyVisits,  txtVisits, p_sess, ts_sess = Parse.getDaysBehavioralData(datafile, expCode)
clustlist, clustQuals = util.getClustList(datafile) # clustList and clustQuals share same order. ith entry in each is the name and qual of same cell. 
population = OrderedDict()

unit = Core.UnitData(clustname, datafile, expCode, Def.alleyBounds, alleyVisits, txtVisits, p_sess, ts_sess)
unit.loadData_raw()

E:\UserData\Documents\GitHub\ratterdam\Beltway_Project\ratterdam_CoreDataStructures.py:178: RuntimeWarning: divide by zero encountered in reciprocal
  n = (hs*np.reciprocal(ho))*30
E:\UserData\Documents\GitHub\ratterdam\Beltway_Project\ratterdam_CoreDataStructures.py:178: RuntimeWarning: invalid value encountered in multiply
  n = (hs*np.reciprocal(ho))*30
E:\UserData\Documents\GitHub\ratterdam\Beltway_Project\ratterdam_CoreDataStructures.py:184: RuntimeWarning: divide by zero encountered in reciprocal
  n = (ls* np.reciprocal(lo)) * 30
E:\UserData\Documents\GitHub\ratterdam\Beltway_Project\ratterdam_CoreDataStructures.py:184: RuntimeWarning: invalid value encountered in multiply
  n = (ls* np.reciprocal(lo)) * 30
E:\UserData\Documents\GitHub\ratterdam\utility_fx.py:330: RuntimeWarning: invalid value encountered in true_divide
  Z=VV/WW


### Read in tetrode csc data into data and ts structs

In [28]:
tt = 5
fname = datafile + f"CSC{tt}.ncs"

curFile = open(fname)
curFile.seek(16*1024)
curFile = np.fromfile(curFile, dtype=ncsType)

dt = (1/32000)*1e6 # this is the timestep between each sample in packet. Only first sample is explicitly timestamped
                     # by neuralynx at time of collection so increment this dt value repeatedly to get ts of all other samples in packet
cscdata = curFile['data'].flatten()
cscts = []
for packet in curFile:
    init_ts = packet['ts']
    cscts.append(init_ts)
    interp_ts = init_ts + dt # initialize the, basically, counter. this init is for the second point if that makes sense
    for datapoint in packet['data'][1:]: # start with second bc first one is (the only one) timestamped
        cscts.append(interp_ts)
        interp_ts += dt # increment for next ts
        
csc = np.column_stack((cscts, cscdata))
del cscts, cscdata

### Extract Data

In [29]:
#whole session tt theta will be kept 
theta = np.column_stack((csc[:,0],filter_data(csc[:,1],8,12)))

In [10]:
def extractCSCSegment(csc, tsBeg, tsEnd):
    """
    Input - * csc is (n,2) array with whole session csc data
            for one tt (sampled 1/32000 s). ts, sample value
            * tsBeg, tsEnd - neuralynx ts for segment
            start and end
    Return: 
    """
    mask = (csc[:,0]>tsBeg)&(csc[:,0]<tsEnd)
    segment = csc[mask,:]
    return segment

In [32]:
def calcPrecessionTrial(spikes, thetaSegment, coordDict):
    
    #poormans point projection is just take coordinate of long dimension of alley
    if coordDict[alley] == 'y':
        spos = spikes[:,2]
    elif coordDict[alley] == 'x':
        spos = spikes[:,1]
    
    phaseSegment = np.column_stack((thetaSegment[:,0], np.angle(hilbert3(thetaSegment[:,1]), deg=True)))
    precess = np.column_stack((phaseSegment[np.searchsorted(phaseSegment[:,0], spikes[:,0])-1,1],spos))
    
    return precess

In [33]:
alley=8
precession = {'A': np.empty((0,2)), 
              'B': np.empty((0,2)), 
              'C': np.empty((0,2))}

print(f"Unit {unit.name}, alley {alley}")

for visit in unit.alleys[alley]:
    spk = visit['spikes']
    txt = visit['metadata']['stimulus']
    tsBeg, tsEnd = visit['occs'][0,0], visit['occs'][-1,0]
    thetaSegment = theta[(theta[:,0]>tsBeg)&(theta[:,0]<tsEnd)]
    precession[txt] = np.vstack((precession[txt], calcPrecessionTrial(spk, thetaSegment, alleylongcoord)))

Unit TT5cl-maze1.1, alley 8


In [35]:
for txt in ['A', 'B', 'C']:
    plt.figure()
    plt.title(f"{rat} {expCode} {clustname} {alley} {txt}")
    plt.scatter(precession[f'{txt}'][:,1], precession[f'{txt}'][:,0])